In [ ]:
import re
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os, glob
import time
import pandas as pd
import numpy as np
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)

In [ ]:
vzorec_bloka = re.compile(
    r'<td><mtg-set-icon.*?'
    r'</tr',
    flags=re.DOTALL
)

vzorec_karte = re.compile(
    r'<td><mtg-set-icon.*?>.*?<i _ngcontent.*? class="ss ss-fw ss-(?P<set>.+?)\b( ss-.*?)?"></i><!----></mtg-set-icon>'
    r'<a href="/prints/(?P<id_karte>\d*).*?">'
    r'(?P<ime>.*?)'
    r'</a>.*?</td>.*?<td> (?P<redkost>.*?) </td>'
    r'.*?<td class="text-end">'
    r' (?P<povprecna_cena>.*?) </td>(<\!---->)+(<td class="text-end">.*?</td>.*?<td class="text-end"> (?P<povprecna_cena_foil>.*?) </td>.*?<td class="text-end">.*?</td>)?',  
    flags=re.DOTALL
    )

def izloci_podatke_o_kartah(blok):
    karta = vzorec_karte.search(blok).groupdict()
    karta['id_karte'] = int(karta['id_karte'])
    karta['ime'] = karta['ime'].strip()
    karta['set'] = karta['set'].upper()
    karta['redkost'] = karta["redkost"].strip()
    karta['povprecna_cena'] = karta["povprecna_cena"]
    karta['povprecna_cena_foil'] = karta["povprecna_cena_foil"]

    return karta

def izloci_podatke_o_specificni_karti(vsebina):
    """
    To bo izločilo vse dodatne podatke o karti, ki jih bomo našli na naslovu .../prints/<id_karte>
    """
    pass

def pridobi_ustrezno_ime_lokalne_datoteke(st_strani):
    return f"Podatki o kartah/Podatki iz setov/Karte iz seta st. {st_strani}.html"

In [ ]:
# Najprej poradiramo csv in json datoteki
with open("karte.csv", "w", encoding = "utf-8") as dat:
    pass

with open("karte.json", "w", encoding = "utf-8") as dat:
    pass

PRVI_SET = 1
ZADNJI_SET = 1300

#indeksi_setov_ki_obstajajo = []

for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
    url = (
        f'https://www.mtgstocks.com/sets/{st_strani}' 
    )
    print(f"\n Zajemam {url}\n")
    driver.get(url)
    time.sleep(1)
    
    
    vsebina = driver.page_source
    naslov_strani = driver.title
    
    stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
    st_besed_Market_na_strani = vsebina.count(">Market<")
    
    print("Naslov strani:", naslov_strani)
    
    if naslov_strani in ["File not found - MTGStocks"]:
        print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
    else:
        print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
        if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
            continue
        else:
            with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w', encoding = "utf-8") as dat:
                driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                    driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                    time.sleep(1)
                    print(f"Pobrana {podstran}. podstran")
                    dat.write(driver.page_source)
  

Koda za zapisovanje kart iz lokalnih datotek v csv in json format.


In [ ]:
debug_mode_za_branje_iz_lokalnih_datotek = False
path = 'Podatki o kartah/Podatki iz setov'
karte = []

for filename in glob.glob(os.path.join(path, '*.html')):
    with open(filename, 'r', encoding = "utf-8") as f:

        stetje = 0
        print(f"Berem podatke kart iz {filename}")
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            if debug_mode_za_branje_iz_lokalnih_datotek:
                print(stetje)
                print(blok.group(0))
                print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izloci_podatke_o_kartah(blok.group(0)))

with open("karte.json", "a", encoding = "utf-8") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a", encoding = "utf-8") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

Zdaj se znebimo kart, ki so tokeni ali basic landi.
Prav tako bi se radi znebili kart, ki imajo tako ceno kot ceno foila ničelno. (Torej da lahko karte primerjamo po vsaj eni komponenti).
Pri tem je vredno omeniti, da imajo karte, ki so najbolj zaželene, tudi najbolj popolne podatke, torej jih s tem ne bomo izgubili.


In [ ]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki = okrajsani_podatki[~(okrajsani_podatki.povprecna_cena.isnull() & okrajsani_podatki.povprecna_cena_foil.isnull())]
okrajsani_indeksi = okrajsani_podatki.id_karte
print(okrajsani_podatki)
print(okrajsani_indeksi)

Zdaj pa pogledamo vsako posamezno karto. Zato bomo naredili posamezne vzorce, ki nam bodo pomagali pri parsanju.
Nesmiselno je imeti samo en vzorec, saj je to samo logistična nočna mora.


In [144]:
vzorec_povprecja_eu = re.compile(
    r'Europe avg </p> (?P<povprecje_eu>.*?) <',
    flags=re.DOTALL
)

vzorec_typea = re.compile(
    r'class="card-oracle-item".*?</i> (?P<type>.*?) <',
    flags=re.DOTALL
)

vzorec_oracle_texta = re.compile(
    r'<.*?class="card-oracle-item card-oracle-text">(?P<oracle_text>.*?)</p>',
    flags=re.DOTALL
)

vzorec_all_time_low = re.compile(
    r'All Time Low </p><h3.*?>(?P<all_time_low>.*?)</h3><h5.*?> (?P<datum_all_time_low>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_all_time_high = re.compile(
    r'All Time High </p><h3.*?>(?P<all_time_high>.*?)</h3><h5.*?> (?P<datum_all_time_high>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_reserved_list = re.compile(
    r'Reserved list\?</p><h3.*?>(?P<reserved_list>.*?)</h3>',
    flags=re.DOTALL
)

def izlusci_podatke_iz_glavnine(niz):
    povprecje_eu = re.findall(vzorec_povprecja_eu, niz)
    type = re.findall(vzorec_typea, niz)
    reserved_list = re.findall(vzorec_reserved_list, niz)
    
    if len(povprecje_eu) == 1:
        povprecje_eu = povprecje_eu[0]
    else:
        povprecje_eu = None
    if len(type) == 1:
        type = type[0]
    else:
        type = None
    if len(reserved_list) == 1:
        reserved_list = reserved_list[0]
    else:
        reserved_list = None
    
    najdi_vse_low = re.findall(vzorec_all_time_low, niz)
    najdi_vse_high = re.findall(vzorec_all_time_high, niz)
    if len(najdi_vse_low) == 1:
        if len(najdi_vse_low[0]) == 2:
            all_time_low, all_time_low_datum = najdi_vse_low[0][0], najdi_vse_low[0][1]
    else:
        all_time_low, all_time_low_datum = None, None
    if len(najdi_vse_high) == 1:
        if len(najdi_vse_high[0]) == 2:
            all_time_high, all_time_high_datum = najdi_vse_high[0][0], najdi_vse_high[0][1]
    else:
        all_time_high, all_time_high_datum = None, None
    return {"povprecje_eu": povprecje_eu, "type": type, "reserved_list":reserved_list, "all_time_low": all_time_low, "all_time_low_datum": all_time_low_datum, "all_time_high": all_time_high, "all_time_high_datum": all_time_high_datum}



In [160]:
vzorec_za_manacost = re.compile(r'" class="ms ms-cost ms-(?P<mana_cost>.*?)["| ]',
                                flags=re.DOTALL)

def izlusci_podatke_manacosta_in_barve(niz):
    seznam_simbolov = re.findall(vzorec_za_manacost, niz)
    cmc = 0
    manacost = ""
    barva = ""
    for simbol in seznam_simbolov:
        manacost += simbol
        if simbol.isnumeric():
            cmc += int(simbol)
        elif simbol in "wubrg":
            cmc += 1
            if simbol not in barva:
                barva += simbol
        else:
            if simbol not in "{(p)}x":
                cmc += 1
            if simbol not in "{(p)}c":
                print("Program je našel nenavaden simbol", simbol)
    
    # To je majhen popravek za karte s hybrid manacostom
    # Potreben je, ker je to bug na strani MTGSrocks (narobno zapisuje hybrid cost)
    cmc -= barva.count("(") + barva.count("{")
    
    if barva == "":
        barva += "c"
    if manacost == "":
        manacost += "0"
    return {"cmc": cmc, "manacost": manacost, "barva": barva}

with open("testni_zajcek.txt", "r", encoding = "utf-8") as testna_datoteka:
    vsebina = testna_datoteka.read()
    
glavnina = izlusci_podatke_iz_glavnine(vsebina)
manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
manacost_in_barve.update(glavnina)
print(manacost_in_barve)


{'cmc': 0, 'manacost': '0', 'barva': 'c', 'povprecje_eu': '€12.64', 'type': 'Land - Island Mountain', 'reserved_list': 'No', 'all_time_low': '$6.79', 'all_time_low_datum': 'Apr 6, 2013', 'all_time_high': '$21.18', 'all_time_high_datum': 'Jul 20, 2022'}


In [163]:
print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))

# To je toliko da poradiramo
with open("specificne_lastnosti_kart.csv", "w", encoding = "utf-8") as csvfile:
    pass

with open("specificne_lastnosti_kart.csv", "a", encoding = "utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, [
        "id_karte",
        "cmc",
        "manacost",
        "barva",
        "povprecje_eu",
        "type",
        "reserved_list",
        "all_time_low",
        "all_time_low_datum",
        "all_time_high",
        "all_time_high_datum",
    ])
    
    podatki_o_kartah = []

    for indeks_printa in okrajsani_indeksi[500:1500]:
        url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
        print(f"Zajemanje printa karte št. {indeks_printa} s strani {url}")
        # with open(f"Podatki o kartah/Podatki o specificnih kartah/Podatki o karti st. {indeks_printa}.html", "w", encoding="utf-8") as dat:
        try:
            driver.get(url)
            time.sleep(1)
            vsebina = driver.page_source
            
            glava_slovarja = {"id_karte": indeks_printa}
            manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
            glavnina = izlusci_podatke_iz_glavnine(vsebina)
            glava_slovarja.update(manacost_in_barve)
            glava_slovarja.update(glavnina)
            print(glava_slovarja, "\n")
            podatki_o_kartah.append(glava_slovarja)
            
        except Exception as e:
            print(f"Prišlo je do napake pri setu {indeks_printa}")
            print(e)
            
    writer.writeheader()
    writer.writerows(podatki_o_kartah)
        

Število vseh indeksov kart, ki jih imamo: 43057
Zajemanje printa karte št. 75064 s strani https://www.mtgstocks.com/prints/75064


C:\Users\jasak\AppData\Local\Temp\ipykernel_20420\2296832594.py:24: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for indeks_printa in okrajsani_indeksi[500:1500]:


{'id_karte': 75064, 'cmc': 4, 'manacost': '3w', 'barva': 'w', 'povprecje_eu': '€1.62', 'type': 'Creature - Rhino Druid, ', 'reserved_list': 'No', 'all_time_low': '$2.75', 'all_time_low_datum': 'May 1, 2022', 'all_time_high': '$9.80', 'all_time_high_datum': 'May 3, 2022'} 

Zajemanje printa karte št. 74942 s strani https://www.mtgstocks.com/prints/74942
{'id_karte': 74942, 'cmc': 1, 'manacost': '1', 'barva': 'c', 'povprecje_eu': '€9.00', 'type': 'Artifact', 'reserved_list': 'No', 'all_time_low': '$2.45', 'all_time_low_datum': 'May 3, 2022', 'all_time_high': '$7.97', 'all_time_high_datum': 'May 13, 2022'} 

Zajemanje printa karte št. 74945 s strani https://www.mtgstocks.com/prints/74945
{'id_karte': 74945, 'cmc': 3, 'manacost': '3', 'barva': 'c', 'povprecje_eu': '€4.27', 'type': 'Artifact Creature - Wall, ', 'reserved_list': 'No', 'all_time_low': '$0.63', 'all_time_low_datum': 'Jun 1, 2022', 'all_time_high': '$6.85', 'all_time_high_datum': 'Oct 13, 2022'} 

Zajemanje printa karte št. 750

In [111]:
string_za_poskus = ''
print(re.findall(vzorec_za_manacost, string_za_poskus))

print(izlusci_podatke_manacosta_in_barve(string_za_poskus))

[]
{'cmc': 0, 'manacost': '0', 'barva': 'c'}
